In [2]:
import nltk
nltk.download("treebank")

[nltk_data] Downloading package treebank to
[nltk_data]     /home/santiago/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
from nltk.corpus import treebank
from nltk.tag.hmm import HiddenMarkovModelTrainer
from nltk.tag import tnt, CRFTagger
from nltk.tag.perceptron import PerceptronTagger

data = treebank.tagged_sents()
HMMtrainer = HiddenMarkovModelTrainer()
TnT = tnt.TnT()
PER = PerceptronTagger(load=False)
CRF = CRFTagger()
sizes = [500, 1000, 1500, 2000, 2500, 3000]
accuracies = {'HMM': [], 'TnT': [], 'PER': [], 'CRF': []}

for s in sizes:
    HMM = HMMtrainer.train_supervised(data[:s])
    accuracies['HMM'].append(round(HMM.evaluate(data[3001:]), 3))
    
    TnT.train(data[:s])
    accuracies['TnT'].append(round(TnT.evaluate(data[3001:]), 3))
    
    PER.train(data[:s])
    accuracies['PER'].append(round(PER.evaluate(data[3001:]), 3))
    
    CRF.train(data[:s], 'crf_tagger_model')
    accuracies['CRF'].append(round(CRF.evaluate(data[3001:]), 3))

print(accuracies)

{'HMM': [0.173, 0.221, 0.262, 0.304, 0.335, 0.369], 'TnT': [0.747, 0.796, 0.827, 0.848, 0.862, 0.875], 'PER': [0.911, 0.933, 0.942, 0.947, 0.951, 0.955], 'CRF': [0.91, 0.925, 0.933, 0.939, 0.943, 0.948]}


In [4]:
import matplotlib.pyplot as plt
for a in accuracies:
    plt.plot(sizes, accuracies[a])
plt.xlabel('Training data size')
plt.ylabel('Accuracy')
plt.legend(accuracies.keys())
plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
# We would choose the perceptron since it is the one
# that performs better regardless of the training data size

In [6]:
#SMS Spam filter from lab3:

import numpy as np
import string

# load data
data = np.loadtxt('smsspamcollection/SMSSpamCollection', dtype='str',delimiter='\t')

# pre process
#remove punctuation and lower chars
translator = str.maketrans('', '', string.punctuation)
for i in range(data.shape[0]):
    data[i,1] = data[i,1].translate(translator).lower()
    
#Single validation 50/50
train = data[:2787]
test = data[2787:]

#random shuffle
np.random.shuffle(train)
np.random.shuffle(test)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric, NearestNeighbors
from sklearn.svm import SVC
from nltk.metrics import ConfusionMatrix
from nltk.metrics.scores import accuracy

cv = CountVectorizer()
tfcv = TfidfVectorizer()
knn = KNeighborsClassifier(1)
svc = SVC(kernel='linear')

vectorizers = [[cv, 'CountVectorizer'],[tfcv,'TfidfVectorizer']]
classifiers = [[knn,'knn'], [svc,'svc']]

#Classifiers with KNN and SVM with linear kernel
#Also using two vectorizers Count and Tfid
#The best combination seems to be the SVM with Tfid vectorizer, but 
#they all obtain a high accuracy
for v in vectorizers:
    vectorizer = v[0]
    Xtrn = vectorizer.fit_transform([''.join(ex[1]) for ex in train])
    Xtst = vectorizer.transform([''.join(ex[1]) for ex in test])
    Ytrn = [ex[0] for ex in train]
    Ytst = [ex[0] for ex in test]
    for classifier in classifiers:
        clf = classifier[0]
        clf.fit(Xtrn, Ytrn)
        preds = clf.predict(Xtst).tolist()
        print('Results for classifier: ',classifier[1], ' with vectorizer: ', v[1])
        print('Accuracy score: ', round(accuracy(test[:,0], preds), 3))
        print(ConfusionMatrix(test[:,0].tolist(), preds).pretty_format())
    

Results for classifier:  knn  with vectorizer:  CountVectorizer
Accuracy score:  0.94
     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<2420>   1 |
spam |  165 <201>|
-----+-----------+
(row = reference; col = test)

Results for classifier:  svc  with vectorizer:  CountVectorizer
Accuracy score:  0.978
     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<2413>   8 |
spam |   53 <313>|
-----+-----------+
(row = reference; col = test)

Results for classifier:  knn  with vectorizer:  TfidfVectorizer
Accuracy score:  0.975
     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<2412>   9 |
spam |   61 <305>|
-----+-----------+
(row = reference; col = test)

Results for classifier:  svc  with vectorizer:  TfidfVectorizer
Accuracy score:  0.981
     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<24

In [8]:
from nltk.metrics.distance import jaccard_distance

def kNN(ex, d):
    return min(trnlist, key=lambda x: d(ex[1], x[1]))[0]


def jaccard_d(a,b):                                       
    return jaccard_distance(set(a), set(b))

In [9]:
# KNN with jaccard distance

import nltk

trnlist = train.tolist()
tstlist = test.tolist()

for i in range(len(train)):
    sentences = nltk.sent_tokenize(train[i][1])
    words = [nltk.word_tokenize(sent) for sent in sentences]
    trnlist[i][1] = words[0]
    
for i in range(len(test)):
    sentences = nltk.sent_tokenize(train[i][1])
    words = [nltk.word_tokenize(sent) for sent in sentences]
    tstlist[i][1] = words[0]

preds = []
refs = []
for tst in tstlist:
    preds.append(kNN(tst, jaccard_d))
    refs.append(tst[0])
    
print('Accuracy score: ', round(accuracy(refs, preds), 3))
print(ConfusionMatrix(refs, preds).pretty_format())

Accuracy score:  0.761
     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<2081> 340 |
spam |  325  <41>|
-----+-----------+
(row = reference; col = test)



In [10]:
def lemmatize(p):
    if p[1][0] in {'N', 'V'}:
        return wnl.lemmatize(p[0].lower(), pos=p[1][0].lower())
    return p[0]

In [13]:
# KNN with lemmas

from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

trnlist = train.tolist()
tstlist = test.tolist()

for i in range(len(train)):
    sentences = nltk.sent_tokenize(train[i][1])
    words = [nltk.word_tokenize(sent) for sent in sentences]
    pairs = [pos_tag(w) for w in words]
    l_words = [[lemmatize(p) for p in pair] for pair in pairs]
    trnlist[i][1] = l_words[0]
    
for i in range(len(test)):
    sentences = nltk.sent_tokenize(train[i][1])
    words = [nltk.word_tokenize(sent) for sent in sentences]
    pairs = [pos_tag(w) for w in words]
    l_words = [[lemmatize(p) for p in pair] for pair in pairs]
    tstlist[i][1] = l_words[0]
    
preds = []
refs = []
for tst in tstlist:
    preds.append(kNN(tst, jaccard_d))
    refs.append(tst[0])
    
print('Accuracy score: ', round(accuracy(refs, preds), 3))
print(ConfusionMatrix(refs, preds).pretty_format())

Accuracy score:  0.761
     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<2081> 340 |
spam |  325  <41>|
-----+-----------+
(row = reference; col = test)

